In [ ]:
from routingapp.common.routing_utils import *

In [13]:
link_quality_controller = rq.get('http://0.0.0.0:8080/link_quality').json()
link_quality_mininet = rq.get('http://0.0.0.0:8000/link_quality').json()
link_ping_stat = rq.get('http://0.0.0.0:8000/link_ping_stat').json()

link_ping_stat

[{'src.host': 1,
  'dst.host': 5,
  'packet_loss': 0.03902439024390244,
  'delay': 82.054},
 {'src.host': 5,
  'dst.host': 1,
  'packet_loss': 0.03902439024390244,
  'delay': 82.054},
 {'src.host': 1,
  'dst.host': 2,
  'packet_loss': 0.07804878048780488,
  'delay': 130.039},
 {'src.host': 2,
  'dst.host': 1,
  'packet_loss': 0.07804878048780488,
  'delay': 130.039},
 {'src.host': 5,
  'dst.host': 4,
  'packet_loss': 0.0902439024390244,
  'delay': 70.043},
 {'src.host': 4,
  'dst.host': 5,
  'packet_loss': 0.0902439024390244,
  'delay': 70.043},
 {'src.host': 2,
  'dst.host': 6,
  'packet_loss': 0.03170731707317073,
  'delay': 78.042},
 {'src.host': 6,
  'dst.host': 2,
  'packet_loss': 0.03170731707317073,
  'delay': 78.042},
 {'src.host': 3,
  'dst.host': 4,
  'packet_loss': 0.08048780487804878,
  'delay': 96.043},
 {'src.host': 4,
  'dst.host': 3,
  'packet_loss': 0.08048780487804878,
  'delay': 96.043},
 {'src.host': 1,
  'dst.host': 3,
  'packet_loss': 0.07804878048780488,
  'delay

In [5]:
_, graph = get_topo()
host_json = get_host()
link_info = get_link_quality()

{'hosts': [{'mac': '00:00:00:00:0b:b8',
   'ipv4': [],
   'ipv6': ['::', 'fe80::200:ff:fe00:bb8'],
   'port': {'dpid': '0000000000000003',
    'port_no': '00000002',
    'hw_addr': '9a:74:62:a9:42:d1',
    'name': 's3-eth2'}},
  {'mac': '00:00:00:00:03:e8',
   'ipv4': [],
   'ipv6': ['::', 'fe80::200:ff:fe00:3e8'],
   'port': {'dpid': '0000000000000001',
    'port_no': '00000002',
    'hw_addr': '7e:cc:b6:ca:e0:ca',
    'name': 's1-eth2'}},
  {'mac': '00:00:00:00:17:70',
   'ipv4': [],
   'ipv6': ['::', 'fe80::200:ff:fe00:1770'],
   'port': {'dpid': '0000000000000006',
    'port_no': '00000002',
    'hw_addr': 'd2:24:03:88:27:bf',
    'name': 's6-eth2'}},
  {'mac': '00:00:00:00:00:03',
   'ipv4': [],
   'ipv6': ['::', 'fe80::200:ff:fe00:3'],
   'port': {'dpid': '0000000000000003',
    'port_no': '00000001',
    'hw_addr': '4e:ac:db:60:17:3f',
    'name': 's3-eth1'}},
  {'mac': '00:00:00:00:07:d0',
   'ipv4': [],
   'ipv6': ['::', 'fe80::200:ff:fe00:7d0'],
   'port': {'dpid': '000000000

In [14]:
_, graph = get_topo()
host_json = get_host()
link_info = get_link_quality()

# Add host to graph
for host in host_json['hosts']:
    dpid_int = mac_to_int(host['port']['dpid'])
    host_int = mac_to_int(host['mac'])
    # print(f'dpid_int: {dpid_int}, host_int: {host_int}')
    
    # Add node to graph
    graph.add_node(f'h{host_int}', type='host')
    # add bi-directional link between host and switch
    graph.add_edge(f'h{host_int}', dpid_int, type='host')
    graph.add_edge(dpid_int, f'h{host_int}', type='host')

# print graph to json
# print(nx.node_link_data(graph))

# Mapping host h{int} to int
mapping = dict(zip(graph.nodes(), range(1, len(graph.nodes())+1)))
print(mapping)
# Creating adj-matrix of graph
number_node = len(graph.nodes())
bin_matrix = nx.adjacency_matrix(graph).todense()
adj_matrix = [[] for i in range(number_node+1)]
for i in range(1, number_node+1):
    for j in range(1, number_node+1):
        if bin_matrix[i-1][j-1] == 1:
            adj_matrix[i].append(j)

# Get from data from /link_quality
update_delay = []
update_link_utilization = []
update_loss = []
for stat in link_info:
    src = mapping[stat['src.dpid']]
    dst = mapping[stat['dst.dpid']]
    if src != dst:
        delay = stat.get('delay', 0)
        if delay == None: delay = 0
        loss = stat.get('packet_loss', 0)
        if loss == None: loss = 0
        bandwidth = stat.get('link_utilization', 0)
        if bandwidth == None: bandwidth = 0
        update_delay.append((src, dst, delay))
        update_loss.append((src, dst, loss))
        update_link_utilization.append((src, dst, bandwidth))
    
# # Reading request
# routes = task.route
# request = []

# for route in routes:
#     src = f'h{route.src_host}'
#     dst = f'h{route.dst_host}'
#     src = mapping[src]
#     dst = mapping[dst]
#     print('reading rq', src, dst)
#     request.append((src, dst))


{4: 1, 3: 2, 2: 3, 5: 4, 6: 5, 1: 6, 'h6000': 7, 'h2': 8, 'h5000': 9, 'h1000': 10, 'h2000': 11, 'h3000': 12, 'h4000': 13, 'h4': 14, 'h6': 15, 'h5': 16, 'h1': 17, 'h3': 18}


In [21]:
update_loss

[(1, 4, 0.0878048780487805),
 (1, 2, 0.08292682926829269),
 (2, 6, 0.07560975609756097),
 (2, 1, 0.08292682926829269),
 (3, 5, 0.03170731707317073),
 (3, 6, 0.07317073170731708),
 (4, 1, 0.0878048780487805),
 (4, 6, 0.03902439024390244),
 (4, 5, 0.07560975609756097),
 (5, 3, 0.03170731707317073),
 (5, 4, 0.07560975609756097),
 (6, 4, 0.03902439024390244),
 (6, 2, 0.07560975609756097),
 (6, 3, 0.07317073170731708)]

In [19]:
a: list[tuple] = [(1,2), (3,4)]

In [10]:
from routingapp.common.routing_utils import get_network_stat_legacy
type(get_network_stat_legacy()[1][0])

KeyError: 0